#### Import Functions

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#### Data Reading

In [0]:
df_products = spark.readStream.format('delta') \
                    .load('abfss://bronze@dldatabricks.dfs.core.windows.net/products')

In [0]:
# display(df_products)

In [0]:
df_products = df_products.drop('_rescued_data')

#### **Functions**

In [0]:
df_products.createOrReplaceTempView('products')

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_catalog.bronze.discount_func(p_price DOUBLE)
RETURNS DOUBLE
LANGUAGE SQL
RETURN round(p_price * 0.90,4)

In [0]:
%sql
SELECT *, databricks_catalog.bronze.discount_func(price) AS price_after_discount FROM products

In [0]:
from pyspark.sql.functions import expr

df_products = df_products.withColumn('price_after_discount', expr('databricks_catalog.bronze.discount_func(price)'))
df_products.display()

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_catalog.bronze.upper_func(p_brand STRING)
RETURNS STRING
LANGUAGE PYTHON
AS 
$$ 
    return p_brand.upper()
$$

In [0]:
%sql
SELECT *, databricks_catalog.bronze.upper_func(brand) AS brand_upper FROM products

#### Data Writing

In [0]:
df_products.writeStream.format('delta') \
            .outputMode('append') \
            .option('checkpointLocation', 'abfss://silver@dldatabricks.dfs.core.windows.net/checkpoint_products') \
            .option('path','abfss://silver@dldatabricks.dfs.core.windows.net/products') \
            .trigger(once=True) \
            .start()